In [1]:
from tardis.montecarlo.montecarlo_numba.opacities import *
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from astropy import units as u
timing_dict = {}

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)


In [2]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
tardis_config['montecarlo']['iterations'] = 1
sim = Simulation.from_config(tardis_config)

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)


In [3]:
electron_density = sim.plasma.electron_densities[0]

In [4]:
%%timeit

calculate_tau_electron(electron_density, 5.0)

260 ns ± 6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
timing_dict["calculate_tau_electron"]= (196*u.ns*25263789).to(u.ms)
(196*u.ns*25263789).to(u.ms)

<Quantity 4951.702644 ms>

In [6]:
from tardis.montecarlo.montecarlo_numba.r_packet import RPacket
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line
base_rpacket = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890, 0, False)
rpacket1 = base_rpacket

In [7]:
%%timeit

calculate_distance_line(rpacket1, 912784333619689.2, False, 912500130829976.2, 1123200.0)

490 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
timing_dict["calculate_distance_line"] = (557*u.ns*25263789).to(u.ms)
(557*u.ns*25263789).to(u.ms)

<Quantity 14071.930473 ms>

In [9]:
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_electron
electron_density = sim.plasma.electron_densities[0]
import numpy as np
tau_event = -np.log(np.random.random())

In [10]:
%%timeit

calculate_distance_electron(electron_density, tau_event)

288 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
timing_dict["calculate_distance_electron"] = (231*u.ns*25263789).to(u.ms)
(231*u.ns*25263789).to(u.ms)

<Quantity 5835.935259 ms>

In [12]:
from tardis.montecarlo.montecarlo_numba.r_packet import test_for_close_line
from tardis.montecarlo.montecarlo_numba.numba_interface import numba_plasma_initialize
rpacket2 = base_rpacket
base_numba_plasma = numba_plasma_initialize(sim.plasma, "macroatom")
plasma1 = base_numba_plasma

In [13]:
%%timeit

test_for_close_line(rpacket2, 1, .1, plasma1)

720 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [14]:
timing_dict["test_for_close_line"] = (743*u.ns*23432925).to(u.ms)
(743*u.ns*23432925).to(u.ms)

<Quantity 17410.663275 ms>

In [15]:
from tardis.montecarlo.montecarlo_numba.estimators import update_line_estimators
from tardis.montecarlo.montecarlo_numba.numba_interface import Estimators
base_j_estimator = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
base_nu_bar_estimator = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
base_j_blue_estimator = np.zeros([29224, 20])
base_Edotlu_estimator = np.zeros([29224, 20])
base_estimators = Estimators(base_j_estimator, base_nu_bar_estimator, base_j_blue_estimator, base_Edotlu_estimator)
estimators1 = base_estimators
rpacket3 = base_rpacket
cur_line_id = 1
base_distance_trace = calculate_distance_line(rpacket1, 5.9, False, .1, 10)
distance_trace1 = base_distance_trace
time_explosion = 10

In [16]:
%%timeit

update_line_estimators(estimators1, rpacket3, cur_line_id, distance_trace1, time_explosion)

833 ns ± 191 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
timing_dict["update_line_estimators"] = (908*u.ns*23432925).to(u.ms)
(908*u.ns*23432925).to(u.ms)

<Quantity 21277.0959 ms>

In [18]:
from tardis.montecarlo.montecarlo_numba.frame_transformations import calc_packet_energy
rpacket4 = base_rpacket
distance_trace2 = base_distance_trace

In [19]:
%%timeit

calc_packet_energy(rpacket4, distance_trace2, 10)

441 ns ± 9.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [20]:
timing_dict["calc_packet_energy"] = (460*u.ns*23432925).to(u.ms)
(460*u.ns*23432925).to(u.ms)

<Quantity 10779.1455 ms>

In [21]:
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor
r1 = base_rpacket.r
mu1 = base_rpacket.mu

In [22]:
%%timeit

get_doppler_factor(r1, mu1, 10)

196 ns ± 20.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [23]:
timing_dict["get_doppler_factor"] = (187*u.ns* 4130777).to(u.ms)
(187*u.ns* 4130777).to(u.ms)

<Quantity 772.455299 ms>

In [24]:
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity
from tardis.montecarlo.montecarlo_numba.numba_config import C_SPEED_OF_LIGHT
inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / 10
beta = base_rpacket.r * inv_t * inv_c
mu2 = base_rpacket.mu

In [25]:
%%timeit

get_doppler_factor_partial_relativity(mu2, beta)

240 ns ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
timing_dict["get_doppler_factor_partial_relativity"] = (164*u.ns* 4130777).to(u.ms)
(164*u.ns* 4130777).to(u.ms)

<Quantity 677.447428 ms>

In [28]:
from tardis.montecarlo.montecarlo_numba.r_packet import trace_packet
from tardis.montecarlo.montecarlo_numba.numba_interface import NumbaModel
rpacket0 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model0 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 1123200.0)
numba_plasma0 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
nu_bar_estimator = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
j_blue_estimator = np.zeros([29224, 20])
Edotlu_es0timator = np.zeros([29224, 20])
estimator0 = Estimators(j_estimator, nu_bar_estimator, j_blue_estimator, Edotlu_estimator)
rpacket0.initialize_line_id(numba_plasma0, numba_model0)
print(rpacket0.next_line_id)

9427


In [30]:
%%timeit

trace_packet(rpacket0, numba_model0, numba_plasma0, estimator0)

The slowest run took 9.11 times longer than the fastest. This could mean that an intermediate result is being cached.
5.16 µs ± 5.96 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
timing_dict["trace_packet"] = (2080*u.ns* 4130777).to(u.ms)
print((2080*u.ns* 4130777).to(u.ms))

8592.01616 ms


In [ ]:
"""
    print("RPacket:")
    print(r_packet.r, r_packet.mu, r_packet.nu, r_packet.energy, r_packet.seed)
    print("NumbaModel")
    print(numba_model.r_inner, numba_model.r_outer, numba_model.time_explosion)
    print("ESTIMATORS")
    print(estimators.j_estimator)
    print(estimators.nu_bar_estimator)
    print(estimators.j_blue_estimator)
    print(estimators.j_blue_estimator.size)
    print(estimators.j_blue_estimator.shape)
    print(estimators.Edotlu_estimator)
    print(estimators.Edotlu_estimator.size)
    print(estimators.Edotlu_estimator.shape)"""

In [32]:
sim.plasma.atomic_data.lines

line_id  \
atomic_number ion_number level_number_lower level_number_upper            
14            5          0                  36                    67813   
                                            35                    67814   
                                            34                    67815   
                         1                  35                    67816   
                         0                  33                    67817   
...                                                                 ...   
20            0          176                191                  104458   
                         177                191                  104459   
                         178                191                  104460   
                         138                144                  104461   
                         177                190                  104462   

                                                                   wavelength  \
atomic_number ion_number level_number_lower level_number_upper                  
14            5          0                  36                      66.772000   
                                            35                      69.204000   
                                            34                      69.236000   
                         1                  35                      69.448000   
                         0                  33                      71.181000   
...                                                                       ...   
20            0          176                191                 992063.502847   
                         177                191                 992555.837031   
                         178                191                 993541.985801   
                         138                144                 997804.837623   
                         177                190                 999900.018655   

                                                                    f_ul  \
atomic_number ion_number level_number_lower level_number_upper             
14            5          0                  36                  0.027030   
                                            35                  0.086684   
                                            34                  0.061923   
                         1                  35                  0.017699   
                         0                  33                  0.091590   
...                                                                  ...   
20            0          176                191                 0.033180   
                         177                191                 0.024882   
                         178                191                 0.001655   
                         138                144                 3.380648   
                         177                190                 0.000265   

                                                                    f_lu  \
atomic_number ion_number level_number_lower level_number_upper             
14            5          0                  36                  0.040545   
                                            35                  0.086684   
                                            34                  0.092884   
                         1                  35                  0.035397   
                         0                  33                  0.137385   
...                                                                  ...   
20            0          176                191                 0.099541   
                         177                191                 0.024882   
                         178                191                 0.000993   
                         138                144                 1.126883   
                         177                190                 0.000441   

                                                        

In [34]:
numba_plasma0.line_list_nu.shape

(29224,)